Installing transformer's langchain


In [6]:
!pip install -q transformers langchain accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 16.2 MB/s eta 0:00:00


Logging in for using its libraries and models. This needs a hugging face token that is assosciated with the Hugging face account

In [7]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

Importing all the necessary libraries

In [8]:
import torch
import transformers
from transformers import AutoTokenizer
from  langchain import LLMChain, HuggingFacePipeline, PromptTemplate

We use Llama 2 as our base model and then get the tokenizer needed to tranform the text for the model we are using.
An analogy: Think of it like translating the text into a language that Llama 2 speaks well.


In [9]:
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Using the 'tranformer.pipeline' function for creating a pipeline for text generation and setting all the parameters. (Taking max length as 10000 to handle large texts)

In [10]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    max_length=10000,
    device_map = 'auto',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Creating a HuggingFacePipeline by passing in the pipeline variable as its parameter created in the previous step. We need to create a hugging face pipeline in order to use models from HuggingFace's transformer library.

Temperature is a parameter that controls the randomness of the sampling process in text generation. A value of 0 means that the model will always choose the most probable token at each step, resulting in deterministic text generation.

In [11]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

Defining the prompt template for the model

In [12]:
template = """
              Write a detailed summary of the following text delimited by triple backticks,
              please ensure that the summary captures the main ideas, arguments, and findings presented in the text. As the text is lengthy and will be processed in batches, the summary should seamlessly integrate information from all batches to provide a cohesive summary of the entire content.
              ```{text}```
              SUMMARY:
           """

creating a prompt using the 'PromptTemplate' that is used used to create a template for prompting the language model.

LLMChain : Chain to run queries against LLMs, which takes promt and the the pipileine we created for text generation using Llama2 as input

In [13]:
prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [14]:
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 1.9 MB/s eta 0:00:00


In [15]:
# importing required modules
from pypdf import PdfReader

# creating a pdf reader object
reader = PdfReader('flairs-36-140.pdf')

Extracting the text from the pdf

In [95]:
text = ''
for i in range(len(reader.pages)):
    page = reader.pages[i]
    text_temp = page.extract_text()
    text = text + "\n" + text_temp

In [96]:
len(text)

13768

Removing the reference section from the text

In [17]:
def remove_references(text):
    common_patterns = ["References"]

    # Find the index of the earliest occurrence of the common pattern
    start_index = len(text)
    for pattern in common_patterns:
        index = text.find(pattern)
        if index != -1 and index < start_index:
            start_index = index

    # Remove everything after the start of the references section
    if start_index < len(text):
        text = text[:start_index]

    return text

# Remove the references section
cleaned_text = remove_references(text)
print(cleaned_text)



A Survey on Sentiment Classification Methods and Challenges
Neda Keivandarian, Marco Carvalho
Dept. of Engineering and Sciences Florida Institute of Technology
Melbourne, USA
nkeivandaria2020@my.fit.edu, mcarvalho@fit.edu
Abstract
Sentiment classification (SC) is an important natural
language processing (NLP) task that aims to determine
the sentiment or emotional tone in a given text. With
the increasing pervasiveness of internet-based applica-
tions and social media, massive amounts of unstructured
data are generated daily, elevating the opportunity and
challenges associated with automated sentiment extrac-
tion for tasks such as customer feedback analysis, social
media monitoring, and opinion mining. In this review
paper, we provide an update on the state of the art in
sentiment analysis, including an overview of and classi-
fication methods leveraging machine learning and deep
learning methods.
Keywords —Sentiment classification, Machine learn-
ing, Survey, Text pre-processing
Intr

In [18]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [19]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2, 3'

In [20]:
import nltk
nltk.download('punkt')

def get_segments(text):
  sentences = nltk.sent_tokenize(text)
  return sentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Dividing the text into batches, due to memory errors.

In [21]:
def divide_into_batches(text,batch_size):
  segments = get_segments(text)

  batches = []
  current_batch = []

  for segment in segments:

    if len(" ".join(current_batch)) + len(segment) > batch_size:
      batches.append(current_batch)
      current_batch = []

    current_batch.append(segment)


  if current_batch:
    batches.append(current_batch)

  return batches

def summarize_batches(batches):
  summaries = []

  for batch in batches:
    batch_text = " ".join(batch)

    summary = llm_chain.run(batch_text)

    summaries.append(summary)

  return summaries


In [22]:
batch_size = 3500
batches = divide_into_batches(cleaned_text, batch_size) # cleaned text - without references
summaries = summarize_batches(batches)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [23]:
len(summaries)

3

In [24]:
# import re

# # Given string
# text = '''\n              Write a detailed summary of the following text delimited by triple backticks.\n              Return your response which covers the key points of the text.\n              ```\nA Survey on Sentiment Classification Methods and Challenges\nNeda Keivandarian, Marco Carvalho\nDept. of Engineering and Sciences Florida Institute of Technology\nMelbourne, USA\nnkeivandaria2020@my.fit.edu, mcarvalho@fit.edu\nAbstract\nSentiment classification (SC) is an important natural\nlanguage processing (NLP) task that aims to determine\nthe sentiment or emotional tone in a given text. With\nthe increasing pervasiveness of internet-based applica-\ntions and social media, massive amounts of unstructured\ndata are generated daily, elevating the opportunity and\nchallenges associated with automated sentiment extrac-\ntion for tasks such as customer feedback analysis, social\nmedia monitoring, and opinion mining. In this review\npaper, we provide an update on the state of the art in\nsentiment analysis, including an overview of and classi-\nfication methods leveraging machine learning and deep\nlearning methods.```\n              SUMMARY:\n            This paper provides an overview of the current state of sentiment classification (SC) in natural language processing (NLP). The authors discuss the increasing pervasiveness of internet-based applications and social media, which has led to a massive amount of unstructured data being generated daily. They highlight the opportunities and challenges associated with automated sentiment extraction, including customer feedback analysis, social media monitoring, and opinion mining. The authors then provide an overview of and classification methods leveraging machine learning and deep learning methods. They also discuss the challenges associated with sentiment classification, including the lack of annotated datasets, the complexity of natural language, and the need for better interpretability of machine learning models. The authors conclude by highlighting the need for further research in this area to improve the accuracy and interpretability of sentiment classification models.'''

# # Regular expression pattern to match text after "SUMMARY:"
# pattern = r'SUMMARY:(.*)'

# # Extract text after "SUMMARY:"
# match = re.search(pattern, text, re.DOTALL)
# if match:
#     summary = match.group(1).strip()
#     print(summary)
# else:
#     print("No summary found")


In [25]:
import re

def extract_summary(text):

    # Regular expression pattern to match text after "SUMMARY:"
  pattern = r'SUMMARY:(.*)'

    # Extract text after "SUMMARY:"
  match = re.search(pattern, text, re.DOTALL)
  if match:
      return match.group(1).strip()
  else:
      return "No summary found"


final_summary = []
# Apply function to each string in the list
for text in summaries:
  # print(text)
  summary = extract_summary(text)
  final_summary.append(summary)


In [26]:
final_summary

['The article "A Survey on Sentiment Classification Methods and Challenges" discusses the current state of sentiment classification, including machine learning and deep learning approaches. The authors provide an overview of various methods for sentiment classification at different levels, including document, sentence, and aspect levels. They also highlight the challenges associated with sentiment classification, such as dealing with imbalanced datasets and handling sarcasm and irony. The authors emphasize the need for further research in this field to improve the accuracy of sentiment classification models.',
 'The article discusses various approaches to sentiment classification, including machine learning and deep learning methods. The authors highlight the importance of pre-processing raw text data to remove noise and improve accuracy. They also discuss various techniques for feature extraction, including tokenization, normalization, stop word removal, part of speech tagging, and st

In [27]:
combined_summary = " ".join(final_summary)

In [28]:
combined_summary

'The article "A Survey on Sentiment Classification Methods and Challenges" discusses the current state of sentiment classification, including machine learning and deep learning approaches. The authors provide an overview of various methods for sentiment classification at different levels, including document, sentence, and aspect levels. They also highlight the challenges associated with sentiment classification, such as dealing with imbalanced datasets and handling sarcasm and irony. The authors emphasize the need for further research in this field to improve the accuracy of sentiment classification models. The article discusses various approaches to sentiment classification, including machine learning and deep learning methods. The authors highlight the importance of pre-processing raw text data to remove noise and improve accuracy. They also discuss various techniques for feature extraction, including tokenization, normalization, stop word removal, part of speech tagging, and stemmin

In [29]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [30]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=447ecffa1d6c45115eede7595e0aa5f8870d49889345970590abd4ef5192c8fe
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [44]:
human_written_summary = 'The survey paper delves into the rapidly increasing challenges and opportunities presented by the exponential growth of internet-based applications and social media, leading to the generation of vast unstructured data. This growth leads to the generation of vast unstructured data. The paper aims to provide a comprehensive overview of recent state-of-the-art approaches in sentiment classification (SC), encompassing various techniques and methodologies. It categorizes sentiment levels into document, sentence, and aspect levels, providing an overview of related work within each. The approach section outlines the entire process of sentiment classification, from data collection and pre-processing to feature extraction and model development. It provides insights into the best practices for each stage in the process. The paper outlines three model development approaches: machine learning, deep learning, and transfer learning, providing insights into algorithms and related work for each approach. Challenges in sentiment classification, such as irony, sarcasm, and noise in unstructured data, are discussed. Proposed solutions like a voting classifier gradient boosted SVM and considerations for adversarial attacks are also mentioned. In conclusion, the paper emphasizes the effectiveness of machine learning methods like SVM and Naive Bayes. However, it highlights the preference for deep learning approaches such as LSTM and BERT due to their adeptness in handling complex problems with large datasets. The paper notes that sentiment classification remains an evolving area of study. It suggests future research avenues for expanding the comparison of deep learning methods. The authors advocate for further exploration of these methods to advance the field. The study concludes by underscoring the need for ongoing research and development in sentiment classification.'

In [45]:
ref = get_segments(human_written_summary)

In [46]:
ref

['The survey paper delves into the rapidly increasing challenges and opportunities presented by the exponential growth of internet-based applications and social media, leading to the generation of vast unstructured data.',
 'This growth leads to the generation of vast unstructured data.',
 'The paper aims to provide a comprehensive overview of recent state-of-the-art approaches in sentiment classification (SC), encompassing various techniques and methodologies.',
 'It categorizes sentiment levels into document, sentence, and aspect levels, providing an overview of related work within each.',
 'The approach section outlines the entire process of sentiment classification, from data collection and pre-processing to feature extraction and model development.',
 'It provides insights into the best practices for each stage in the process.',
 'The paper outlines three model development approaches: machine learning, deep learning, and transfer learning, providing insights into algorithms and re

In [34]:
!pip install rogue

  Preparing metadata (setup.py) ... done
  Created wheel for rogue: filename=rogue-0.0.2-py3-none-any.whl size=7210 sha256=83ed532cc263c2ff27ce7f27469179244473197e33f3c054ff20edadcce59f62
  Stored in directory: /root/.cache/pip/wheels/11/db/ee/a44bd5d88bd4903f31c1f677a28ac79e354555bca48332f232
Successfully built rogue


In [149]:
print('Performance based on Rogue score: ')

Performance based on Rogue score: 


In [48]:
from evaluate import load
# Load the ROUGE metric
import evaluate
rouge = evaluate.load('rouge')
candidates = get_segments(combined_summary)
# print(len(candidates))
references = []
references.append(ref)
# print(len(references[0])) #it should have more refernce summaries in order to analyze the summary generated by llm
results = rouge.compute(predictions=candidates, references=references[0])
print(results)

{'rouge1': 0.1732908370911774, 'rouge2': 0.01763668430335097, 'rougeL': 0.14810922558140271, 'rougeLsum': 0.14608063084817047}


I think, the generated summary was good enough, and the Rogue scores above are not the correct represenation of the comparision.

It should have more summaries to compare, but due to lack of time I could not generate different refernce summaries to compare. I think having more reference summaries to compare might have resulted into better score.

Q2- passing in the whole text without removing the reference section, instead explicitly mentioning that in the promt template.

template = """
              Write a detailed summary of the following text delimited by triple backticks,
              please ensure that the summary captures the main ideas, arguments, and findings presented in the text. As the text is lengthy and will be processed in batches, the summary should seamlessly integrate information from all batches to provide a cohesive summary of the entire content.
              ```{text}```
              SUMMARY:
           """

In [105]:
template2 = """
              Write a detailed summary of the following text delimited by triple backticks,
              please ensure that the summary captures the main ideas, arguments, and findings presented in the text. As the text is lengthy and will be processed in batches, the summary should seamlessly integrate information from all batches to provide a cohesive summary of the entire content.
              Ignore the references section if it appears in any batch of the text.
              ```{text}```
              SUMMARY:
           """

In [106]:
prompt2 = PromptTemplate(template=template2, input_variables=["text"])
llm_chain2 = LLMChain(prompt=prompt2, llm=llm)

In [107]:
def divide_into_batches(text,batch_size):
  segments = get_segments(text)

  batches = []
  current_batch = []

  for segment in segments:

    if len(" ".join(current_batch)) + len(segment) > batch_size:
      batches.append(current_batch)
      current_batch = []

    current_batch.append(segment)


  if current_batch:
    batches.append(current_batch)

  return batches

def summarize_batches(batches):
  summaries = []

  for batch in batches:
    batch_text = " ".join(batch)

    summary = llm_chain2.run(batch_text)  # defining the function again with llm_chain2

    summaries.append(summary)

  return summaries


In [108]:
text = ''
for i in range(len(reader.pages)):
    page = reader.pages[i]
    text_temp = page.extract_text()
    text = text + "\n" + text_temp

In [111]:
len(text)

13768

In [114]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [115]:
batch_size = 1500
batches = divide_into_batches(text, batch_size) # text - with references
summaries2 = summarize_batches(batches)

In [116]:
len(summaries2)

10

In [150]:
# summaries2

In [118]:
import re

def extract_summary(text):

    # Regular expression pattern to match text after "SUMMARY:"
  pattern = r'SUMMARY:(.*)'

    # Extract text after "SUMMARY:"
  match = re.search(pattern, text, re.DOTALL)
  if match:
      return match.group(1).strip()
  else:
      return "No summary found"


final_summary2 = []
# Apply function to each string in the list
for text in summaries2:
  # print(text)
  summary = extract_summary(text)
  final_summary2.append(summary)


In [119]:
final_summary2

['This survey provides an update on the state of the art in sentiment analysis, focusing on classification methods leveraging machine learning and deep learning techniques. The authors review various approaches, including feature extraction, classification algorithms, and deep learning models, and highlight the challenges associated with sentiment classification. They also provide an overview of the existing survey on sentiment analysis, including text pre-processing techniques and the importance of sentiment classification in various applications. The authors conclude by emphasizing the need for further research to improve the accuracy and efficiency of sentiment classification methods.',
 'This paper focuses on comprehensive and recent approaches in sentiment analysis, particularly in the field of Subjective Commentary (SC). The authors aim to provide a detailed overview of various methods, including document-level sentiment classification, sentence-level sentiment classification, an

In [120]:
combined_summary2 = " ".join(final_summary2)

In [137]:
human_written_summary2 = 'The survey paper delves into the rapidly increasing challenges and opportunities presented by the exponential growth of internet-based applications and social media, leading to the generation of vast unstructured data. This growth leads to the generation of vast unstructured data. The paper aims to provide a comprehensive overview of recent state-of-the-art approaches in sentiment classification (SC), encompassing various techniques and methodologies. It categorizes sentiment levels into document, sentence, and aspect levels, providing an overview of related work within each. The approach section outlines the entire process of sentiment classification, from data collection and pre-processing to feature extraction and model development. It provides insights into the best practices for each stage in the process. The paper outlines three model development approaches: machine learning, deep learning, and transfer learning, providing insights into algorithms and related work for each approach. Challenges in sentiment classification, such as irony, sarcasm, and noise in unstructured data, are discussed. Proposed solutions like a voting classifier gradient boosted SVM and considerations for adversarial attacks are also mentioned. In conclusion, the paper emphasizes the effectiveness of machine learning methods like SVM and Naive Bayes. However, it highlights the preference for deep learning approaches such as LSTM and BERT due to their adeptness in handling complex problems with large datasets. The paper notes that sentiment classification remains an evolving area of study. It suggests future research avenues for expanding the comparison of deep learning methods. '

In [141]:
human_written_summary2 = human_written_summary2*6   # to make it the same length as the generated summary

adding one more sentence

In [144]:
new_sentence = 'It also notes that sentiment classification remains an evolving area of study, suggesting future research avenues for expanding the comparison of deep learning methods.'

In [145]:
human_written_summary2 = human_written_summary2 + " " + new_sentence

In [146]:
ref2 = get_segments(human_written_summary2)

In [148]:
from evaluate import load
# Load the ROUGE metric
import evaluate
rouge = evaluate.load('rouge')
candidates = get_segments(combined_summary2)
# print(len(candidates))
references = []
references.append(ref2)
# print(len(references[0])) #it should have more refernce summaries in order to analyze the summary generated by llm
results = rouge.compute(predictions=candidates, references=references[0])
print('Performance based on Rogue score without removing references: ')
print(results)

Performance based on Rogue score without removing references: 
{'rouge1': 0.12641146196552178, 'rouge2': 0.009422437278817771, 'rougeL': 0.10190180542513848, 'rougeLsum': 0.10157436087067259}
